# Rag implementation

In [18]:
import time
import os
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate


load_dotenv()

pinecone_api_key = os.getenv('PINECONE_API_KEY')
openai_api_key = os.getenv("OPENAI_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

index_name = "hackathon-divhacks"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]



index = pc.Index(index_name)

# Step 2: Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings(
    openai_api_key=openai_api_key,
    model="text-embedding-ada-002"  # Specify the embedding model
)

# Step 3: Setup Pinecone as the vector store for Langchain

vector_store = PineconeVectorStore(index=index, embedding=embeddings)
# Step 4: Memory to track previous conversations
memory = ConversationBufferMemory()

# Step 5: Setup Langchain with Conversational Retrieval Chain
rag_chain = ConversationalRetrievalChain(
    retriever=vector_store.as_retriever(search_kwargs={"k": 3}),  # Number of documents to retrieve
    memory=memory,
    prompt_template=PromptTemplate(
        input_variables=["context", "question"],
        template="""
        You are an emotional AI companion. Use the following context and answer the user's question empathetically:

        Context:
        {context}

        User's Question: {question}
        """
    )
)

# Step 6: Function to interact with the chatbot
def ask_chatbot(user_input):
    response = rag_chain.run(question=user_input)
    return response

# Example usage: Store and retrieve conversations
user_input_1 = "I've been feeling really anxious lately."
response_1 = ask_chatbot(user_input_1)
print(f"Bot Response: {response_1}")

# Simulate another conversation with context retrieval
user_input_2 = "Can you help me with some techniques for managing anxiety?"
response_2 = ask_chatbot(user_input_2)
print(f"Bot Response: {response_2}")



/var/folders/6n/_3v7211x5hj3mg4qs5wn4yph0000gn/T/ipykernel_67081/3247711458.py:37: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/var/folders/6n/_3v7211x5hj3mg4qs5wn4yph0000gn/T/ipykernel_67081/3247711458.py:40: LangChainDeprecationWarning: The class `ConversationalRetrievalChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~create_history_aware_retriever together with create_retrieval_chain (see example in docstring)` instead.
  rag_chain = ConversationalRetrievalChain(


ValidationError: 3 validation errors for ConversationalRetrievalChain
combine_docs_chain
  Field required [type=missing, input_value={'retriever': VectorStore... {question}\n        ")}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
question_generator
  Field required [type=missing, input_value={'retriever': VectorStore... {question}\n        ")}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
prompt_template
  Extra inputs are not permitted [type=extra_forbidden, input_value=PromptTemplate(input_vari...: {question}\n        "), input_type=PromptTemplate]
    For further information visit https://errors.pydantic.dev/2.9/v/extra_forbidden